<a href="https://colab.research.google.com/github/beryaelio/chatbot/blob/main/chatbot_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simple chatbot using the NLTK and sklearn python libraries.
The data is from a Wikipedia page about Artificial Neural Network:
https://en.wikipedia.org/wiki/Artificial_neural_network  


In [1]:
import numpy as np
import nltk
import string
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
from google.colab import drive
drive.mount('/content/drive')

f = open('/content/drive/MyDrive/chatbot/artificial_neural_network_wiki_data.txt', 'r', errors = 'ignore')
raw_data = f.read()

Mounted at /content/drive


After I've opened the data, I'll tokenize it.

In [3]:
raw_data = raw_data.lower()
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
sentence_tokens = nltk.sent_tokenize(raw_data)
word_tokens = nltk.word_tokenize(raw_data)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


After tokenizing I'll lemmetize the tokens.

In [4]:
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
  return [lemmer.lemmatize(token) for token in tokens]
remove_punt_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
  return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punt_dict)))

I make a function for the greeting as well. So the bot will be polite:)

In [5]:
greet_input = ('hello', 'hi', 'how are you?')
greet_response = ('hi', 'hey')
def greet(sentence):
  for word in sentence.split():
    if word.lower() in greet_input:
      return random.choice(greet_response)

Now in here is where the magic happens. Based on the user's input, using the feature extraction method: Term Frequency-Inverse Document Frequency Vectorizer and the cosine similarity metric, the bot will find text in the data that most likely the user queries about. Based on similarity between the user input and the data, the bot will either return no similarity found therefore another query is needed or a subtext that most probably matches the user input.

In [6]:
def response(user_response):
  robo_response = ''
  TfidfVec = TfidfVectorizer(tokenizer = LemNormalize, stop_words = 'english')
  Tfidf = TfidfVec.fit_transform(sentence_tokens)
  vals = cosine_similarity(Tfidf[-1], Tfidf)
  idx = vals.argsort()[0][-2]
  flat = vals.flatten()
  flat.sort()
  reqTfidf = flat[-2]
  if (reqTfidf == 0):
    robo_response = robo_response + "Sorry I can't understand you please try again"
    return robo_response
  else:
    robo_response = robo_response + sentence_tokens[idx]
    return robo_response

This the last part, which is the functionality of the bot. Based on the user's input, the code will determine whether to return a greeting, a response or exit the run.

In [7]:
flag = True
print('Hello, start typing your text after greeting to talk to me. For ending convo type bye!')
while(flag == True):
  user_response = input()
  user_response = user_response.lower()
  if(user_response != 'bye!'):
    if(user_response == 'Thanks' or user_response == 'Thank you'):
      flag = False
      print('Happy to help!')
    else:
      if(greet(user_response) != None):
        print('Bot: '+ greet(user_response))
      else:
        sentence_tokens.append(user_response)
        word_tokens = word_tokens + nltk.word_tokenize(user_response)
        final_words = list(set(word_tokens))
        print('Bot: ', end = '')
        print(response(user_response))
        sentence_tokens.remove(user_response)
  else:
    flag = False
    print('Bot: Goodbye!')

Hello, start typing your text after greeting to talk to me. For ending convo type bye!
hi
Bot: hey
I want to know about neural networks
Bot: 

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


artificial neural networks (anns, also shortened to neural networks (nns) or neural nets) are a branch of machine learning models that are built using principles of neuronal organization discovered by connectionism in the biological neural networks constituting animal brains.
are there models?
Bot: the first network is a generative model that models a probability distribution over output patterns.
what else can you tell me about artificial functions 
Bot: artificial neurons
anns are composed of artificial neurons which are conceptually derived from biological neurons.
ok thank you, bye!
Bot: Sorry I can't understand you please try again
bye!
Bot: Goodbye!


It's a very simple chatbot, the quality of the answers depends on the quality of the data and any upgrades that can be done to the code.
